## Viral taxonomy on pig vOTUs

in sourmash/viral_taxonomy

Do with both ICTV + genbank 

Steps:
1) run a fastmultigather of all vOTUs against a viral taxonomy db. 
all vOTUs in: sig_files/signatures_concat/allvOTUs.zip
2) do at different scales, 100 and 1000, see what the difference is between the 2. 
3) Use snakefile to go from fastmultigather to taxonomy. Update sourmash, see if now no normal gather needed?

Compare to vcontact2 output

In [ ]:
# use the snake
srun --account=ctbrowngrp -p med2 -J fmg -t 12:00:00 -c 30 --mem=60gb --pty bash
snakemake --resources mem_mb=60000 --rerun-triggers mtime -c 30 --rerun-incomplete -k 

In [ ]:
## genbank
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-viral-k21.rocksdb 

Start with genbank, as file structure is easier:
Taxonomy csv: 
/group/ctbrowngrp/sourmash-db/genbank-2022.03/genbank-2022.03-viral.lineages.csv.gz

sourmash tax genome --gather-csv <gather_csv> [ ... ] --taxonomy-csv

In [ ]:
# srun, activate everything
srun --account=ctbrowngrp -p med2 -J fmg -t 12:00:00 -c 24 --mem=50gb --pty bash

mamba activate branchwater

mkdir genbank_s1000
cd genbank_s1000

# run the fastmultigather
sourmash scripts fastmultigather \
../votu_sigs_s100.txt \
/home/ntpierce/2023-spillover/output.genbank-viral/genbank.2023-05.viral.dna.zip \
-c 24 -k 21 -t 200 -s 100




## Rocksdb not working:
- For the genbank one: Error: Invalid argument: Column family not found: metadata
- For the ictv one: Error: No such file or directory (os error 2)

In [ ]:
srun --account=ctbrowngrp -p med2 -J fmg -t 24:00:00 -c 24 --mem=70gb --pty bash

mamba activate branchwater

# test with rocksdb
# run the fastmultigather
sourmash scripts fastmultigather -c 24 -k 21 -t 3000 -s 1000 -o genbank_s1000.mfg.csv \
votu_sigs_s1000.txt \
/home/ntpierce/2023-spillover/output.genbank-viral/genbank.2023-05.viral.dna-k21-sc10.rocksdb

genbank.2023-05.viral.dna-k21-sc100.rocksdb
# ictv
sourmash scripts fastmultigather -c 24 -k 21 -t 3000 -s 1000 -o ictv_s1000.mfg.csv \
votu_sigs_s1000.txt \
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dna-k21-sc5.rocksdb

# symlink
ln -s /home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dna-k21-sc200.rocksdb .
vmr_MSL38_v1.dna-k21-sc200.rocksdb
# if symlink doesnt work make own rocksdb

In [ ]:
# because vcontact isnt updated with current python, use Chris old version
source /home/csantosm/initconda
conda activate VCONTACT2

# Compare to vcontact2 output
# prodigal file already avail
# remove spaces from the headers..
sed '/^>/ s/ .*//' ../../../virsorter2/protein_files >> tomato_pigeon_br_ns.faa


# and gene2genome
mamba activate vcontact2

python vcontact_gene2genome.py -p 240214_allvOTUs_highq.ns.faa -o \
240214_allvOTUs_highqx.csv -s Prodigal-FAA

# run vcontact2, 24 threads, 70GB, may need more

vcontact2 --raw-proteins 240214_allvOTUs_highq.ns.faa \
--rel-mode 'Diamond' \
--db 'ProkaryoticViralRefSeq85-Merged' \
--proteins-fp 240214_allvOTUs_highq.csv \
--pcs-mode MCL \
--vcs-mode ClusterONE \
--threads 24 \
--c1-bin /home/csantosm/miniconda3/bin/cluster_one-1.0.jar \
--output-dir .


In [ ]:
csvtk in
csvtk concat *.tax.csv > ../ICTV_alltax_s100.csv

## ictv
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dna.zip
vmr_MSL38_v1.dna-k21-sc1.rocksdb  

taxonomy:
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.taxonomy.csv

In [ ]:
# sketch vOTUs at different scales if neccesary
for f in *.fa
do
echo sourmash sketch dna -p k=21,scaled=100 $f --name ${f%.fa*} -o ../../vOTUs_100/${f%.fa*}.sig.gz
done | parallel -j 42
&&
for f in *.fa
do
echo sourmash sketch dna -p k=21,scaled=500 $f --name ${f%.fa*} -o ../../vOTUs_500/${f%.fa*}.sig.gz
done | parallel -j 42